In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.externals import joblib
import cv2
from IPython import display
import signal
import matplotlib.patches as patches
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
class FaceDemo(object):
    def __init__(self, cascade_path):
        self.vc = None
        self.cascade = cv2.CascadeClassifier(cascade_path)
        self.margin = 10
        self.is_interrupted = False
        self.le = None
        self.clf = None
        
    def _signal_handler(self, signal, frame):
        self.is_interrupted = True
        
    def infer(self):
        vc = cv2.VideoCapture(0)
        self.vc = vc
        if vc.isOpened():
            is_capturing, _ = vc.read()
        else:
            is_capturing = False

        signal.signal(signal.SIGINT, self._signal_handler)
        self.is_interrupted = False
        while is_capturing:
            is_capturing, frame = vc.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = self.cascade.detectMultiScale(frame,
                                         scaleFactor=1.1,
                                         minNeighbors=3,
                                         minSize=(100, 100))
            pred = None
            if len(faces) != 0:
                face = faces[0]
                (x, y, w, h) = face
                left = x - self.margin // 2
                right = x + w + self.margin // 2
                bottom = y - self.margin // 2
                top = y + h + self.margin // 2
                img = resize(frame[bottom:top, left:right, :],
                             (160, 160), mode='reflect')
                embs = custom_model.predict(np.expand_dims(img, axis=0))
                pred = self.le.inverse_transform(self.clf.predict(embs))
                cv2.rectangle(frame,
                              (left-1, bottom-1),
                              (right+1, top+1),
                              (255, 0, 0), thickness=2)
            plt.imshow(frame)
            plt.title(pred)
            plt.xticks([])
            plt.yticks([])
            display.clear_output(wait=True)
            try:
                plt.pause(0.1)
            except Exception:
                pass
            if self.is_interrupted:
                vc.release()
                break

In [3]:
custom_model = load_model("./models/saved/mtcnn/custome_FR_Inceptionv1_512D_mtcnn.h5")
knn_classifier = joblib.load("./models/saved/mtcnn/knn.joblib")
encoder = joblib.load("./models/saved/mtcnn/encoder.joblib")
#custom_model.summary()
cascade_path = "./models/haarcascade_frontalface_alt2.xml"
cascade = cv2.CascadeClassifier(cascade_path)

/home/admin/anaconda3/envs/rudra/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
facedlib = FaceDemo(cascade_path)
facedlib.le = encoder
facedlib.clf = knn_classifier
facedlib.infer()

## Instead of using harcascade use MTCNN(multitasking cascading CNN) for face detection

In [5]:
class FaceDemoMTCNN(object):
    def __init__(self, cascade_path):
        self.vc = None
        self.cascade = cv2.CascadeClassifier(cascade_path)
        self.margin = 10
        self.is_interrupted = False
        self.le = None
        self.clf = None
        
    def _signal_handler(self, signal, frame):
        self.is_interrupted = True
        
    def infer(self):
        vc = cv2.VideoCapture(0)
        self.vc = vc
        if vc.isOpened():
            is_capturing, _ = vc.read()
        else:
            is_capturing = False

        signal.signal(signal.SIGINT, self._signal_handler)
        self.is_interrupted = False
        from align_mtcnn import AlignMTCNN
        align_image_mtcnn = AlignMTCNN()
        pnet, rnet, onet = align_image_mtcnn.get_session(0.1)
        while is_capturing:
            is_capturing, frame = vc.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = align_image_mtcnn.detect_face(frame, pnet, rnet, onet, largetsbb=True)
            pred = None
            if len(faces) != 0:
                face = faces[0]
                (x, y, w, h) = face
                left = int(round(x)) - self.margin // 2
                right = int(round(w)) + self.margin // 2
                bottom = int(round(y)) - self.margin // 2
                top =  int(round(h)) + self.margin // 2
                print("xfkjgkjdf", bottom,top, left,right)
                img = resize(frame[bottom:top, left:right, :],
                             (160, 160), mode='reflect')
                embs = custom_model.predict(np.expand_dims(img, axis=0))
                pred = self.le.inverse_transform(self.clf.predict(embs))
                cv2.rectangle(frame,
                              (left-1, bottom-1),
                              (right+1, top+1),
                              (255, 0, 0), thickness=2)
            plt.imshow(frame)
            plt.title(pred)
            plt.xticks([])
            plt.yticks([])
            display.clear_output(wait=True)
            try:
                plt.pause(0.1)
            except Exception:
                pass
            if self.is_interrupted:
                vc.release()
                break

In [ ]:
facemtcnn = FaceDemoMTCNN(cascade_path)
facemtcnn.le = encoder
facemtcnn.clf = knn_classifier
facemtcnn.infer()

In [53]:
import cv2
from sklearn.externals import joblib
import align
import numpy as np
from keras.models import load_model
img_path = "data/test/230220.png"
clf_path = "./models/saved/knn.joblib"
le_path = "./models/saved/encoder.joblib"
face_detector = "models/shape_predictor_68_face_landmarks.dat"

def load_img(img_path):
    img = cv2.imread(img_path, 1)
    return img[..., ::-1]

def align_image(img, face_detector):
    aligndlib = align.AlignDlib(face_detector)
    return aligndlib.align(160, img, aligndlib.getLargestFaceBoundingBox(img),
                               landmarkIndices=align.AlignDlib.OUTER_EYES_AND_NOSE)



/home/admin/anaconda3/envs/rudra/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
model = load_model("models/facenet_keras.h5")
model.load_weights("models/facenet_keras_weights.h5")

In [16]:
# def classify(img_path, clf_path, le_path, model, face_detector):
img = load_img(img_path)
clf = joblib.load(clf_path)
encoder = joblib.load(le_path)
aligned_image = align_image(img, face_detector)
aligned_image = (aligned_image / 255.).astype(np.float32)


In [40]:
np.expand_dims(aligned_image,axis=0).shape

(1, 160, 160, 3)

In [43]:
images = []
for index, i in enumerate(range(10)):
    images.append(aligned_image)
len(images)

10

In [50]:
z = np.asarray(images)



In [101]:
emb = model.predict(z)
# emb = model.predict(np.expand_dims(aligned_image, axis=0))

In [104]:
len(emb.tolist())

10

In [ ]:
emb = model.predict([np.expand_dims(aligned_image, axis=0)])[0]
pred = clf.predict([emb])
prediction = encoder.inverse_transform(pred)
prediction

In [105]:
clf.predict(emb)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [1]:
!which python

/home/admin/anaconda3/bin/python
